In [2]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow

### download and clean data

In [3]:
con_data = pd.read_csv('https://cocl.us/concrete_data')
con_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
con_data.shape

(1030, 9)

In [5]:
con_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
con_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### data cleaned

In [7]:
con_data_column=con_data.columns
X = con_data[con_data_column[con_data_column!= 'Strength']]
y = con_data['Strength']
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### inputs of NN

In [9]:
n_cols= X.shape[1]

# Part C: Increate the number of epochs

### A.1: Split the data

In [10]:
from sklearn.model_selection import train_test_split
# split after, in train_and_test function

### A.2: The regression model

In [11]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

def regression_model(network_arch, n_cols, optimizer='adam', loss='mean_squared_error'):
    '''
    network_arch: 
          {"layer_1": {"n_nodes": n1, "activation": activation1},
           ...
           "layer_n": {"n_nodes: nn}
          }
    '''
    model = Sequential()
    n_layers = len(network_arch)
    output_layer = "layer_"+str(n_layers)
    
    for i,layer in network_arch.items():
        if i == "layer_1":
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation"), input_shape=(n_cols,)))
        elif i == output_layer:
            model.add(Dense(int(layer.get("n_nodes"))))
        else:
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation")))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [12]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

def train_and_test(X, y, network_arch, n_cols, optimizer , loss, n_times, epochs, norm, seed=360):
    '''
    seed: for spilt data
    norm: (bool) normalization or not
    '''
    mse_list=[]
    for i in range(n_times):
        if norm:
            X= preprocessing.StandardScaler().fit(X).transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed+i)
        model = regression_model(network_arch=network_arch, n_cols=n_cols)
        model.fit(X_train, y_train, epochs= epochs, verbose=2)
        y_hat= model.predict(X_test)
        mse = mean_squared_error(y_test, y_hat)
        print("time {} : mse= {}".format(i+1,mse))
        mse_list.append(mse)
    return mse_list

### C.3: Part C result


In [13]:
model_archA = {"layer_1": {"n_nodes": 10, "activation": "relu"}, "layer_2": {"n_nodes": 1},
                }

In [14]:
mse_listA=train_and_test(X=X, y=y, network_arch=model_archA, n_cols=n_cols, optimizer="adam", loss="mean_squared_error", 
                        n_times=50, epochs=100, norm= True, seed=360)

Epoch 1/100
23/23 - 1s - loss: 1565.6584 - 945ms/epoch - 41ms/step
Epoch 2/100
23/23 - 0s - loss: 1549.9939 - 38ms/epoch - 2ms/step
Epoch 3/100
23/23 - 0s - loss: 1535.3677 - 38ms/epoch - 2ms/step
Epoch 4/100
23/23 - 0s - loss: 1521.7120 - 37ms/epoch - 2ms/step
Epoch 5/100
23/23 - 0s - loss: 1508.7122 - 40ms/epoch - 2ms/step
Epoch 6/100
23/23 - 0s - loss: 1496.1302 - 41ms/epoch - 2ms/step
Epoch 7/100
23/23 - 0s - loss: 1483.9182 - 37ms/epoch - 2ms/step
Epoch 8/100
23/23 - 0s - loss: 1471.6455 - 48ms/epoch - 2ms/step
Epoch 9/100
23/23 - 0s - loss: 1459.2344 - 41ms/epoch - 2ms/step
Epoch 10/100
23/23 - 0s - loss: 1446.4894 - 37ms/epoch - 2ms/step
Epoch 11/100
23/23 - 0s - loss: 1433.4443 - 38ms/epoch - 2ms/step
Epoch 12/100
23/23 - 0s - loss: 1419.9088 - 35ms/epoch - 2ms/step
Epoch 13/100
23/23 - 0s - loss: 1405.6222 - 37ms/epoch - 2ms/step
Epoch 14/100
23/23 - 0s - loss: 1391.1418 - 39ms/epoch - 2ms/step
Epoch 15/100
23/23 - 0s - loss: 1375.7056 - 37ms/epoch - 2ms/step
Epoch 16/100
23/2

Epoch 26/100
23/23 - 0s - loss: 965.2879 - 38ms/epoch - 2ms/step
Epoch 27/100
23/23 - 0s - loss: 931.7833 - 38ms/epoch - 2ms/step
Epoch 28/100
23/23 - 0s - loss: 898.4133 - 36ms/epoch - 2ms/step
Epoch 29/100
23/23 - 0s - loss: 865.5098 - 36ms/epoch - 2ms/step
Epoch 30/100
23/23 - 0s - loss: 832.5974 - 36ms/epoch - 2ms/step
Epoch 31/100
23/23 - 0s - loss: 800.3140 - 36ms/epoch - 2ms/step
Epoch 32/100
23/23 - 0s - loss: 768.6339 - 37ms/epoch - 2ms/step
Epoch 33/100
23/23 - 0s - loss: 737.4973 - 40ms/epoch - 2ms/step
Epoch 34/100
23/23 - 0s - loss: 707.4573 - 42ms/epoch - 2ms/step
Epoch 35/100
23/23 - 0s - loss: 678.5329 - 36ms/epoch - 2ms/step
Epoch 36/100
23/23 - 0s - loss: 650.4323 - 36ms/epoch - 2ms/step
Epoch 37/100
23/23 - 0s - loss: 623.6809 - 37ms/epoch - 2ms/step
Epoch 38/100
23/23 - 0s - loss: 598.2141 - 35ms/epoch - 2ms/step
Epoch 39/100
23/23 - 0s - loss: 573.7681 - 34ms/epoch - 1ms/step
Epoch 40/100
23/23 - 0s - loss: 550.5478 - 36ms/epoch - 2ms/step
Epoch 41/100
23/23 - 0s -

Epoch 52/100
23/23 - 0s - loss: 305.9053 - 35ms/epoch - 2ms/step
Epoch 53/100
23/23 - 0s - loss: 294.1397 - 36ms/epoch - 2ms/step
Epoch 54/100
23/23 - 0s - loss: 283.3382 - 35ms/epoch - 2ms/step
Epoch 55/100
23/23 - 0s - loss: 273.5030 - 37ms/epoch - 2ms/step
Epoch 56/100
23/23 - 0s - loss: 264.3581 - 37ms/epoch - 2ms/step
Epoch 57/100
23/23 - 0s - loss: 256.0414 - 38ms/epoch - 2ms/step
Epoch 58/100
23/23 - 0s - loss: 248.3638 - 35ms/epoch - 2ms/step
Epoch 59/100
23/23 - 0s - loss: 241.3591 - 36ms/epoch - 2ms/step
Epoch 60/100
23/23 - 0s - loss: 234.9710 - 36ms/epoch - 2ms/step
Epoch 61/100
23/23 - 0s - loss: 228.9178 - 35ms/epoch - 2ms/step
Epoch 62/100
23/23 - 0s - loss: 223.7074 - 35ms/epoch - 2ms/step
Epoch 63/100
23/23 - 0s - loss: 218.6306 - 37ms/epoch - 2ms/step
Epoch 64/100
23/23 - 0s - loss: 214.0368 - 35ms/epoch - 2ms/step
Epoch 65/100
23/23 - 0s - loss: 209.9201 - 39ms/epoch - 2ms/step
Epoch 66/100
23/23 - 0s - loss: 205.8988 - 37ms/epoch - 2ms/step
Epoch 67/100
23/23 - 0s -

Epoch 78/100
23/23 - 0s - loss: 179.1782 - 36ms/epoch - 2ms/step
Epoch 79/100
23/23 - 0s - loss: 178.0152 - 35ms/epoch - 2ms/step
Epoch 80/100
23/23 - 0s - loss: 176.7618 - 34ms/epoch - 1ms/step
Epoch 81/100
23/23 - 0s - loss: 175.5728 - 35ms/epoch - 2ms/step
Epoch 82/100
23/23 - 0s - loss: 174.4410 - 35ms/epoch - 2ms/step
Epoch 83/100
23/23 - 0s - loss: 173.3050 - 35ms/epoch - 2ms/step
Epoch 84/100
23/23 - 0s - loss: 172.0792 - 34ms/epoch - 1ms/step
Epoch 85/100
23/23 - 0s - loss: 170.9306 - 36ms/epoch - 2ms/step
Epoch 86/100
23/23 - 0s - loss: 169.8156 - 35ms/epoch - 2ms/step
Epoch 87/100
23/23 - 0s - loss: 168.6963 - 35ms/epoch - 2ms/step
Epoch 88/100
23/23 - 0s - loss: 167.5082 - 37ms/epoch - 2ms/step
Epoch 89/100
23/23 - 0s - loss: 166.4053 - 36ms/epoch - 2ms/step
Epoch 90/100
23/23 - 0s - loss: 165.3554 - 39ms/epoch - 2ms/step
Epoch 91/100
23/23 - 0s - loss: 164.3411 - 37ms/epoch - 2ms/step
Epoch 92/100
23/23 - 0s - loss: 163.3293 - 35ms/epoch - 2ms/step
Epoch 93/100
23/23 - 0s -

Epoch 3/100
23/23 - 0s - loss: 1553.7604 - 41ms/epoch - 2ms/step
Epoch 4/100
23/23 - 0s - loss: 1540.1868 - 38ms/epoch - 2ms/step
Epoch 5/100
23/23 - 0s - loss: 1526.6344 - 37ms/epoch - 2ms/step
Epoch 6/100
23/23 - 0s - loss: 1512.4100 - 37ms/epoch - 2ms/step
Epoch 7/100
23/23 - 0s - loss: 1497.8097 - 38ms/epoch - 2ms/step
Epoch 8/100
23/23 - 0s - loss: 1482.1802 - 37ms/epoch - 2ms/step
Epoch 9/100
23/23 - 0s - loss: 1465.6234 - 35ms/epoch - 2ms/step
Epoch 10/100
23/23 - 0s - loss: 1448.2891 - 44ms/epoch - 2ms/step
Epoch 11/100
23/23 - 0s - loss: 1429.8555 - 39ms/epoch - 2ms/step
Epoch 12/100
23/23 - 0s - loss: 1410.2977 - 41ms/epoch - 2ms/step
Epoch 13/100
23/23 - 0s - loss: 1389.4773 - 37ms/epoch - 2ms/step
Epoch 14/100
23/23 - 0s - loss: 1367.4342 - 35ms/epoch - 2ms/step
Epoch 15/100
23/23 - 0s - loss: 1344.1519 - 35ms/epoch - 2ms/step
Epoch 16/100
23/23 - 0s - loss: 1319.8759 - 36ms/epoch - 2ms/step
Epoch 17/100
23/23 - 0s - loss: 1294.9209 - 35ms/epoch - 2ms/step
Epoch 18/100
23/2

...

Epoch 96/100
23/23 - 0s - loss: 173.2520 - 46ms/epoch - 2ms/step
Epoch 97/100
23/23 - 0s - loss: 171.7109 - 43ms/epoch - 2ms/step
Epoch 98/100
23/23 - 0s - loss: 170.1409 - 41ms/epoch - 2ms/step
Epoch 99/100
23/23 - 0s - loss: 168.7069 - 41ms/epoch - 2ms/step
Epoch 100/100
23/23 - 0s - loss: 167.1907 - 43ms/epoch - 2ms/step
time 44 : mse= 152.85910201742024
Epoch 1/100
23/23 - 1s - loss: 1555.3712 - 707ms/epoch - 31ms/step
Epoch 2/100
23/23 - 0s - loss: 1538.8279 - 48ms/epoch - 2ms/step
Epoch 3/100
23/23 - 0s - loss: 1521.8955 - 48ms/epoch - 2ms/step
Epoch 4/100
23/23 - 0s - loss: 1504.0924 - 50ms/epoch - 2ms/step
Epoch 5/100
23/23 - 0s - loss: 1485.2668 - 45ms/epoch - 2ms/step
Epoch 6/100
23/23 - 0s - loss: 1465.4246 - 45ms/epoch - 2ms/step
Epoch 7/100
23/23 - 0s - loss: 1444.5339 - 50ms/epoch - 2ms/step
Epoch 8/100
23/23 - 0s - loss: 1422.7255 - 45ms/epoch - 2ms/step
Epoch 9/100
23/23 - 0s - loss: 1399.3396 - 46ms/epoch - 2ms/step
Epoch 10/100
23/23 - 0s - loss: 1375.2019 - 46ms/epoc

Epoch 21/100
23/23 - 0s - loss: 1284.5723 - 40ms/epoch - 2ms/step
Epoch 22/100
23/23 - 0s - loss: 1259.3859 - 40ms/epoch - 2ms/step
Epoch 23/100
23/23 - 0s - loss: 1233.4584 - 38ms/epoch - 2ms/step
Epoch 24/100
23/23 - 0s - loss: 1206.8488 - 39ms/epoch - 2ms/step
Epoch 25/100
23/23 - 0s - loss: 1179.6017 - 39ms/epoch - 2ms/step
Epoch 26/100
23/23 - 0s - loss: 1152.2043 - 42ms/epoch - 2ms/step
Epoch 27/100
23/23 - 0s - loss: 1123.8827 - 50ms/epoch - 2ms/step
Epoch 28/100
23/23 - 0s - loss: 1095.7781 - 40ms/epoch - 2ms/step
Epoch 29/100
23/23 - 0s - loss: 1067.0837 - 42ms/epoch - 2ms/step
Epoch 30/100
23/23 - 0s - loss: 1038.3362 - 40ms/epoch - 2ms/step
Epoch 31/100
23/23 - 0s - loss: 1009.5029 - 37ms/epoch - 2ms/step
Epoch 32/100
23/23 - 0s - loss: 980.9765 - 39ms/epoch - 2ms/step
Epoch 33/100
23/23 - 0s - loss: 952.0331 - 43ms/epoch - 2ms/step
Epoch 34/100
23/23 - 0s - loss: 923.7888 - 44ms/epoch - 2ms/step
Epoch 35/100
23/23 - 0s - loss: 895.0284 - 45ms/epoch - 2ms/step
Epoch 36/100
2

Epoch 47/100
23/23 - 0s - loss: 267.5042 - 38ms/epoch - 2ms/step
Epoch 48/100
23/23 - 0s - loss: 258.1779 - 36ms/epoch - 2ms/step
Epoch 49/100
23/23 - 0s - loss: 249.9082 - 40ms/epoch - 2ms/step
Epoch 50/100
23/23 - 0s - loss: 242.3448 - 41ms/epoch - 2ms/step
Epoch 51/100
23/23 - 0s - loss: 235.6171 - 42ms/epoch - 2ms/step
Epoch 52/100
23/23 - 0s - loss: 229.4276 - 39ms/epoch - 2ms/step
Epoch 53/100
23/23 - 0s - loss: 224.0223 - 34ms/epoch - 1ms/step
Epoch 54/100
23/23 - 0s - loss: 219.1513 - 38ms/epoch - 2ms/step
Epoch 55/100
23/23 - 0s - loss: 214.7670 - 38ms/epoch - 2ms/step
Epoch 56/100
23/23 - 0s - loss: 210.7608 - 44ms/epoch - 2ms/step
Epoch 57/100
23/23 - 0s - loss: 207.1893 - 53ms/epoch - 2ms/step
Epoch 58/100
23/23 - 0s - loss: 203.8045 - 40ms/epoch - 2ms/step
Epoch 59/100
23/23 - 0s - loss: 200.8967 - 35ms/epoch - 2ms/step
Epoch 60/100
23/23 - 0s - loss: 197.9646 - 37ms/epoch - 2ms/step
Epoch 61/100
23/23 - 0s - loss: 195.4980 - 40ms/epoch - 2ms/step
Epoch 62/100
23/23 - 0s -

Epoch 73/100
23/23 - 0s - loss: 179.2836 - 42ms/epoch - 2ms/step
Epoch 74/100
23/23 - 0s - loss: 177.4254 - 41ms/epoch - 2ms/step
Epoch 75/100
23/23 - 0s - loss: 175.6781 - 43ms/epoch - 2ms/step
Epoch 76/100
23/23 - 0s - loss: 174.0119 - 38ms/epoch - 2ms/step
Epoch 77/100
23/23 - 0s - loss: 172.2058 - 36ms/epoch - 2ms/step
Epoch 78/100
23/23 - 0s - loss: 170.5734 - 36ms/epoch - 2ms/step
Epoch 79/100
23/23 - 0s - loss: 168.9611 - 39ms/epoch - 2ms/step
Epoch 80/100
23/23 - 0s - loss: 167.4228 - 39ms/epoch - 2ms/step
Epoch 81/100
23/23 - 0s - loss: 165.8208 - 41ms/epoch - 2ms/step
Epoch 82/100
23/23 - 0s - loss: 164.3164 - 39ms/epoch - 2ms/step
Epoch 83/100
23/23 - 0s - loss: 162.8563 - 37ms/epoch - 2ms/step
Epoch 84/100
23/23 - 0s - loss: 161.4556 - 37ms/epoch - 2ms/step
Epoch 85/100
23/23 - 0s - loss: 160.1405 - 35ms/epoch - 2ms/step
Epoch 86/100
23/23 - 0s - loss: 158.8087 - 38ms/epoch - 2ms/step
Epoch 87/100
23/23 - 0s - loss: 157.4920 - 36ms/epoch - 2ms/step
Epoch 88/100
23/23 - 0s -

Epoch 99/100
23/23 - 0s - loss: 142.8901 - 44ms/epoch - 2ms/step
Epoch 100/100
23/23 - 0s - loss: 142.1679 - 48ms/epoch - 2ms/step
time 49 : mse= 160.72262593350672
Epoch 1/100
23/23 - 1s - loss: 1662.7076 - 743ms/epoch - 32ms/step
Epoch 2/100
23/23 - 0s - loss: 1646.0941 - 43ms/epoch - 2ms/step
Epoch 3/100
23/23 - 0s - loss: 1630.2542 - 45ms/epoch - 2ms/step
Epoch 4/100
23/23 - 0s - loss: 1615.1117 - 47ms/epoch - 2ms/step
Epoch 5/100
23/23 - 0s - loss: 1600.0300 - 44ms/epoch - 2ms/step
Epoch 6/100
23/23 - 0s - loss: 1585.2062 - 44ms/epoch - 2ms/step
Epoch 7/100
23/23 - 0s - loss: 1570.1206 - 43ms/epoch - 2ms/step
Epoch 8/100
23/23 - 0s - loss: 1554.7562 - 46ms/epoch - 2ms/step
Epoch 9/100
23/23 - 0s - loss: 1539.3147 - 53ms/epoch - 2ms/step
Epoch 10/100
23/23 - 0s - loss: 1523.1097 - 52ms/epoch - 2ms/step
Epoch 11/100
23/23 - 0s - loss: 1506.6077 - 46ms/epoch - 2ms/step
Epoch 12/100
23/23 - 0s - loss: 1489.2996 - 42ms/epoch - 2ms/step
Epoch 13/100
23/23 - 0s - loss: 1471.4794 - 46ms/e

In [17]:
print(mse_listA)

[170.02625741032787, 168.75245558665688, 158.7749644911533, 153.30815624180846, 166.1775421615457, 204.62866657027638, 145.96923129159273, 189.1362373836665, 166.18497697149073, 138.36926569833778, 167.1120142834707, 158.5411957565113, 175.5517747214933, 150.89318650356768, 177.57276463317723, 201.8315204923873, 161.00909242512793, 138.36212505088668, 179.52629346730436, 139.49324907221356, 170.92804125211526, 161.2097914975596, 149.85885134492918, 164.86264202499018, 145.54534201749843, 184.0345422247953, 142.95105458622993, 150.6879257929169, 169.15289709472654, 190.24129125007948, 151.54360230537054, 202.34362281152642, 163.12531395847964, 173.1918186158088, 155.42468299113472, 166.62862241553572, 191.56767106454308, 181.25818862131055, 138.3190248989042, 176.31807928502533, 145.39172909992854, 166.31329215490982, 158.38547829740506, 152.85910201742024, 165.3967380108686, 176.4492391690853, 137.39606683970013, 147.6525959039902, 160.72262593350672, 201.6517136098893]


In [15]:
df_results = pd.DataFrame.from_dict({"Part": ["C"], "mean mse": [np.mean(mse_listA)], "std mse": [np.std(mse_listA)]})
df_results

,Part,mean mse,std mse
0,C,165.052651,17.859949


## Part C result is better than Part B